In [1]:
import tensorflow as tf
import os
import _pickle as cPickle
import numpy as np

CIFAR_DIR = "C:\\Users\\i076453\\Downloads\\tech\\ml\\cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [2]:
# tensorboard
# 1. 指定面板图上显示的变量
# 2. 训练过程中将这些变量计算出来，输出到文件中。
# 3. 文件解析 ./tensorboard --logdir== 将这个文件夹下的文件都进行解析并生成一个面板图

# activation: relu, sigmod, tanh
# weight initializer: he, xavier, normal
# optimizer: Adam, Momentum, Gradient Descent. 
# 以上都是是现在计算图的构建环节


In [3]:
def load_data(filename):
    """ read data from data file """
    with open(filename, 'rb') as f:
        data = cPickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']

# tensorflow.dataset
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)     # 不做filter, 将所有的数据都加载
            all_labels.append(labels)
        self._data = np.vstack(all_data)  # 纵向合并成
        self._data = self._data / 127.5 - 1 # 进行归一化 
            
        self._labels = np.hstack(all_labels) # 横向合并
        
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
        
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch"""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)] 
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

# batch_data, batch_labels = train_data.next_batch(10)
# print(batch_data)
# print(batch_labels)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [4]:
x = tf.placeholder(tf.float32, [None, 3072]) # data 的placeholder, None表示不确定的
# y的shape是 [0.6,5,3]
y = tf.placeholder(tf.int64, [None]) # labels的placeholder

# 转换成多通道
x_image = tf.reshape(x, [-1, 3, 32, 32])
# 交换通道
# 32 x 32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1]) # [-1, 32, 32, 3]

# 封装这些layer
def convnet(inputs, activation, kernel_initializer):
    # 1.13 版本使用的方法跟1.8 的不一样，原有tf.layers里的conv2d等方法作废了
    # https://github.com/tensorflow/tensorflow/issues/26145

    # 构建卷积层
    # conv1 : 神经元图，feature_map, 输出图像
    conv1_1 = tf.keras.layers.Conv2D(
                            32,     # output channel number
                            (3,3),  # kernel size
                            padding = 'same',  # 表示输出输入图像大小一致， 还有一个值是valid
                            activation=activation, # 激活函数
                            kernel_initializer = kernel_initializer,
                            # 默认值None, 但是会使用tf.glorot_uniform_initializer
                            name = 'conv1_1' # 名字
                            )(inputs)
    # 增加卷积层
    conv1_2 = tf.keras.layers.Conv2D(
                            32,     # output channel number
                            (3,3),  # kernel size
                            padding = 'same',  # 表示输出输入图像大小一致， 还有一个值是valid
                            activation=activation, # 激活函数
                            kernel_initializer = kernel_initializer,
                            name = 'conv1_2' # 名字
                            )(conv1_1)

    # 构建池化层
    # 16 x 16
    pooling1 = tf.keras.layers.MaxPool2D(
                                      (2,2), # kernel size
                                      (2,2), # stride 步长 
                                      name = 'pool1')(conv1_2)

    # 构建更多卷积层
    # conv2 : 神经元图，feature_map, 输出图像
    conv2_1 = tf.keras.layers.Conv2D(
                            32,     # output channel numberj
                            (3,3),  # kernel size
                            padding = 'same',  # 表示输出输入图像大小一致， 还有一个值是valid
                            activation=activation, # 激活函数
                            kernel_initializer = kernel_initializer,
                            name = 'conv2_1' # 名字
                            )(pooling1)


    conv2_2 = tf.keras.layers.Conv2D(
                            32,     # output channel numberj
                            (3,3),  # kernel size
                            padding = 'same',  # 表示输出输入图像大小一致， 还有一个值是valid
                            activation=activation, # 激活函数
                            kernel_initializer = kernel_initializer,
                            name = 'conv2_2' # 名字
                            )(conv2_1)

    # 构建更多池化层
    # 8 x 8 
    pooling2 = tf.keras.layers.MaxPool2D(
                                      (2,2), # kernel size
                                      (2,2), # stride 步长 
                                      name = 'pool2')(conv2_2)

    # 构建更多卷积层
    # conv3 : 神经元图，feature_map, 输出图像
    conv3_1 = tf.keras.layers.Conv2D( 
                            32,     # output channel numberj
                            (3,3),  # kernel size
                            padding = 'same',  # 表示输出输入图像大小一致， 还有一个值是valid
                            activation=activation, # 激活函数
                            kernel_initializer = kernel_initializer,
                            name = 'conv3_1' # 名字
                            )(pooling2)

    conv3_2 = tf.keras.layers.Conv2D( 
                            32,     # output channel numberj
                            (3,3),  # kernel size
                            padding = 'same',  # 表示输出输入图像大小一致， 还有一个值是valid
                            activation=activation, # 激活函数
                            kernel_initializer = kernel_initializer,
                            name = 'conv3_2' # 名字
                            )(conv3_1)

    # 构建更多池化层
    # 4 x 4
    pooling3 = tf.keras.layers.MaxPool2D(  
                                      (2,2), # kernel size
                                      (2,2), # stride 步长 
                                      name = 'pool3')(conv3_2)


    # 展平图片 [None, 4 x 4 x 32]
    flatten = tf.keras.layers.Flatten()(pooling3)
    return flatten

# flatten = convnet(x_image, tf.nn.sigmoid) # train 10k: 53.59% 而且sigmoid比relu慢很多
# flatten = convnet(x_image, tf.nn.relu, None)  # train 10k: 73.5%
# truncated_normal_initializer是一个正态分布方法， 可能保证值在一个范围内，truncated表示不会被截断
#  tf.glorot_uniform_initializer： 76.53% for 100k train 
#  truncated_normal_initializer: 78.04% for 100k train
#  tf.keras.initializers.he_normal: 71.52% for 100k train 可能是因为这个网络还不够深
flatten = convnet(x_image, tf.nn.relu, tf.truncated_normal_initializer(stddev=0.02))  # train 10k: 73.5%

# 构建全连接层
y_ = tf.keras.layers.Dense(10)(flatten)


# 交叉熵
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# 它做了下面三件事
# y_ -> softmax
# y -> one hot
# loss = ylogy_

# bool 
predict = tf.argmax(y_, 1) # 在第二个维度求最大值

# [1,0,0,1,0,0,1,1...]
correct_predict = tf.equal(predict, y)

# 准确率
accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float64))

# 有了loss目标函数和准确率后，再去做一个
# AdamOptimizer 是一个剔除向量的变种，1e-3就是10的-3次方0.001 
with tf.name_scope('train_op'):
#     train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)
    train_op = tf.train.GradientDescentOptimizer(le-4).minimize(loss)
    train_op = tf.train.MomentOptimizar(learning_rate=le-4, momentum=0.9).minimize(loss)
# Adam: 78% for train 100k
# Gradient descent：12%
# Moment: 35%
# reason: 1. initializer not good 2. 不充分的训练MomentOptimizar训练的下降没有adam快。 但是最后可能会比adam好
    

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [5]:
# 为了多一些变量，比如：均值，方差，最小最大值等，写一个函数
def variable_summary(var, name):
    """ 给一个变量的很多统计量建立summary """
    with tf.name_scope(name):
        # 求平均值
        mean = tf.reduce_mean(var)
        # 求方差
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        # 构建summary
        tf.summary.scalar('mean', mean)
        tf.summary.scalar('stddev', stddev)
        # 最小值
        tf.summary.scalar('min', tf.reduce_min(var))
        # 最大值
        tf.summary.scalar('max', tf.reduce_max(var))
        # 直方图, 它反映的是变量的分布
        tf.summary.histogram('histogram', var)
        

# 将每个卷积得到的结果输出到summary
with tf.name_scope('summary'):
    variable_summary(conv1_1, 'conv1_1')
    variable_summary(conv1_2, 'conv1_2')
    variable_summary(conv2_1, 'conv2_1')
    variable_summary(conv2_2, 'conv2_2')
    variable_summary(conv3_1, 'conv3_1')
    variable_summary(conv3_2, 'conv3_2')
        
# tensorboard
# 1. 指定面板图上显示的变量  这里我们叫这个过程为“总结”，下面就是分别给损失，正确率进行总结
loss_summary = tf.summary.scalar('loss', loss)
# 'loss': <10,1,1>, <20, 1.08> <次数, loss值>
accuracy_summary = tf.summary.scalar('accuracy', accuracy)

# 首先要将image值进行反归一化，因为之前对它进行了归一化处理，让它的值在-1,1之间
source_image = (x_image + 1) * 127.5

inputs_summary = tf.summary.image('inputs_image', source_image)

# 调用这个api后，会将上面的调用过summary的值都merge起来 
merge_summary = tf.summary.merge_all()

# 还可以显示指定哪些需要merge
merge_summary_test = tf.summary.merge([loss_summary, accuracy_summary])

# 2. 训练过程中将这些变量计算出来，输出到文件中
LOG_DIR = '.'
run_label = 'run_vgg_tensorboard'
# 求出总文件夹
run_dir = os.path.join(LOG_DIR, run_label)
if not os.path.exists(run_dir):
    os.mkdir(run_dir)

train_log_dir = os.path.join(run_dir, 'train')
test_log_dir = os.path.join(run_dir, 'test')
if not os.path.exists(train_log_dir):
    os.mkdir(train_log_dir)
if not os.path.exists(test_log_dir):
    os.mkdir(test_log_dir)
    

In [6]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100

output_summary_every_step = 100


with tf.Session() as sess:
    sess.run(init)
    
    # 创建writer用来写文件到Log目录，用于生成可视化图像
    train_writer = tf.summary.FileWriter(train_log_dir, sess.graph) # 指定计算图的
    test_writer = tf.summary.FileWriter(test_log_dir) # 不指定计算图的  
    
    # 指定一个fixed testset
    fixed_test_batch_data, fixed_test_batch_labels \
        = test_data.next_batch(batch_size)
    
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        
        eval_ops = [loss, accuracy, train_op]
        
        # 只有达到指定输出步的时候才计算这个值
        should_output_summary = ((i+1) % output_summary_every_step == 0)
        if should_output_summary:
            eval_ops.append(merge_summary)
        
        eval_ops_results = sess.run(
            eval_ops,
            feed_dict = {
                x: batch_data,
                y: batch_labels
            }
        )
        loss_val, acc_val = eval_ops_results[0:2]
        if should_output_summary:
            train_summary_str = eval_ops_results[-1]
            # 将train数据插入文件
            train_writer.add_summary(train_summary_str, i+1) # 直接添加，并指定步数
        
            # 将test 数据插入文件
            test_summary_str = sess.run([merge_summary_test], 
                                       feed_dict= {
                                           x: fixed_test_batch_data,
                                           y: fixed_test_batch_labels
                                       })[0]
            test_writer.add_summary(test_summary_str, i+1)
        
        if (i+1) % 500 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' \
                % (i+1, loss_val, acc_val))
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size) 
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels
                    }
                )
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f' % (i+1, test_acc))

[Train] Step: 500, loss: 1.36805, acc: 0.45000
[Train] Step: 1000, loss: 1.39388, acc: 0.50000
[Train] Step: 1500, loss: 1.16307, acc: 0.60000
[Train] Step: 2000, loss: 1.20459, acc: 0.60000
[Train] Step: 2500, loss: 1.25493, acc: 0.55000
[Train] Step: 3000, loss: 1.11048, acc: 0.65000
[Train] Step: 3500, loss: 0.72394, acc: 0.75000
[Train] Step: 4000, loss: 0.55204, acc: 0.85000
[Train] Step: 4500, loss: 0.78430, acc: 0.75000
[Train] Step: 5000, loss: 0.68639, acc: 0.85000
(10000, 3072)
(10000,)
[Test] Step: 5000, acc: 0.66700
[Train] Step: 5500, loss: 0.63437, acc: 0.70000
[Train] Step: 6000, loss: 0.79696, acc: 0.65000
[Train] Step: 6500, loss: 0.99754, acc: 0.70000
[Train] Step: 7000, loss: 0.92484, acc: 0.60000
[Train] Step: 7500, loss: 0.83448, acc: 0.75000
[Train] Step: 8000, loss: 0.68541, acc: 0.80000
[Train] Step: 8500, loss: 0.68005, acc: 0.85000
[Train] Step: 9000, loss: 0.93109, acc: 0.65000
[Train] Step: 9500, loss: 0.22519, acc: 0.95000
[Train] Step: 10000, loss: 0.90532